In [1]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(work_dir+'/fgc_support_retri/')
from fgc_support_retri.train import *
from fgc_support_retri.utils import *
from fgc_support_retri import config

I0324 03:43:00.259097 140093784246080 file_utils.py:39] PyTorch version 1.1.0 available.


In [ ]:
train_sgroup_model(10, 12, "20200324_sgroup_lr=2e-5", 2e-5)

I0324 03:43:02.290840 140093784246080 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0324 03:43:03.186815 140093784246080 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0324 03:43:03.190674 140093784246080 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

dev_set indexing...


 53%|█████▎    | 131/247 [02:10<01:18,  1.48it/s]

# Revise shint

In [10]:
data = json_load(config.FGC_TEST)
get_SHINT(data)
get_answer_sp(data)

100%|██████████| 193/193 [00:00<00:00, 21879.58it/s]

{'text': '不具入侵能力但失控繁殖的细胞，称为良性肿瘤', 'start': 492, 'end': 513}
{'text': '牠们会在夜晚成群的爬出洞穴，循著海浪的声音，利用海面光线的导引，快速的爬向大海的家', 'start': 431, 'end': 472}
{'text': '企鹅是生长在极寒冷的冰天雪地里，被称为不会飞的鸟，牠的翅膀像鱼鳍，能跟海豚一般在碧波里游泳，', 'start': 58, 'end': 104}
{'text': '企鹅是生长在极寒冷的冰天雪地里，被称为不会飞的鸟，牠的翅膀像鱼鳍，能跟海豚一般在碧波里游泳，却不能像海鸟一样飞翔在蔚蓝的天空。', 'start': 58, 'end': 121}


In [11]:
config.FGC_TEST

PosixPath('/work/fgc_support_retri/data/FGC/FGC_release_1.7.8/FGC_release_all_test.json')

In [12]:
json_write(data, 'FGC_release_all_test.json')